In [61]:
#import libraries
import numpy as np
import pandas as pd
import tflearn # tensorflow deep learning
import tensorflow as tf
import random
import json
import pickle
import nltk # natural langauge tool kit for nlp

In [62]:
#nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [63]:
with open('data.json', 'r') as f:
    dataset = json.load(f)

In [64]:
#converting dictinary to pandas dataframe
df = pd.DataFrame.from_dict(dataset['data'])
df.head()

,tag,patterns,responses
0,Fire,"[I am stuck in fire, Fire Fire Fire, Flames al...",[Please consider the following measures- \n 1....
1,Earthquake,"[I am in an earthquake, objects are falling, g...",[Please consider the following measures- \n 1....
2,Flood,"[Lots of water around me, neighbourhood is flo...",[Please consider the following measures- \n 1....


In [65]:
#prepare training data
words = []
labels = []
docs_x = []
docs_y = []

for data in dataset["data"]: # dataset is dict. 'data' is key. dataset['data'] is array of dict. data is dict
    for pattern in data["patterns"]: #pattern is array element in dict element of array of dict
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs_x.append(w)
        docs_y.append(data["tag"])

    if data["tag"] not in labels:
        labels.append(data["tag"])

ignore_words = ['?', '.', '!']
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print(words)

['al', 'am', 'an', 'ar', 'around', 'bed', 'build', 'burn', 'car', 'caught', 'chok', 'crack', 'dam', 'develop', 'drown', 'earthquak', 'everyth', 'fal', 'fil', 'fir', 'flam', 'flood', 'ground', 'heavy', 'high', 'hom', 'hous', 'i', 'in', 'is', 'lan', 'lot', 'me', 'mov', 'my', 'neighbo', 'object', 'of', 'outsid', 'peopl', 'rainfal', 'road', 'room', 'rupt', 'shak', 'smok', 'standstil', 'stuck', 'submerg', 'surround', 'swing', 'the', 'und', 'wat', 'with']


In [66]:
len(words)

55

In [67]:
labels = sorted(labels)
print(labels)

['Earthquake', 'Fire', 'Flood']


In [68]:
print(docs_x)

[['I', 'am', 'stuck', 'in', 'fire'], ['Fire', 'Fire', 'Fire'], ['Flames', 'all', 'around', 'me'], ['Fire', 'all', 'around', 'me'], ['Fire', 'surrounds', 'me'], ['Fire', 'in', 'building'], ['i', 'am', 'caught', 'in', 'fire'], ['Burning', 'my', 'house'], ['high', 'flames', 'burning', 'my', 'car'], ['Smoke', 'is', 'choking', 'me'], ['I', 'am', 'in', 'an', 'earthquake'], ['objects', 'are', 'falling'], ['ground', 'is', 'shaking'], ['Everything', 'is', 'shaking'], ['Ground', 'is', 'moving'], ['The', 'ground', 'is', 'developing', 'cracks'], ['ground', 'is', 'rupturing'], ['Home', 'is', 'damaged'], ['My', 'bed', 'is', 'shaking'], ['The', 'buildings', 'are', 'swinging'], ['Lots', 'of', 'water', 'around', 'me'], ['neighbourhood', 'is', 'flooded'], ['roads', 'submerged', 'under', 'water'], ['house', 'is', 'submerged', 'under', 'water'], ['water', 'all', 'around', 'me'], ['The', 'lanes', 'are', 'choked', 'with', 'water'], ['People', 'are', 'drowning', 'outside'], ['heavy', 'rainfall', 'damaged', '

In [69]:
print(docs_y)

['Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Fire', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Earthquake', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood', 'Flood']


In [70]:
wrds = [stemmer.stem(w) for w in docs_x[13]]
print(wrds)

['everyth', 'is', 'shak']


In [71]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))] # intialise the a list out_empty with 0's with len = length of labels

for x, doc in enumerate(docs_x): # creates tuples of docs_x's index and values such that the index of the docs_x elements are stores in x and the element values are stored in the variable doc
    
	bag = []
    
	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
            
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = np.array(training)
output = np.array(output)

In [72]:
print(training)
print(output)

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [73]:
# neural network model
tf.compat.v1.reset_default_graph() # creates a fresh new empty graph by clearing the nodes and operations of any previous computation

net = tflearn.input_data(shape=[None, len(training[0])]) # None specifies that the input layer can accept a variable number of samples., no of neurons in input layer= 32
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #  no of neurons = 3
net = tflearn.regression(net)
# net = tflearn.softmax_categorical_crossentropy(net,output)
model = tflearn.DNN(net)

In [78]:
# neural network training
try:
	model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")  

Training Step: 3999  | time: 0.008s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 24/30
Training Step: 4000  | time: 0.012s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 30/30
--
INFO:tensorflow:C:\Users\manya\PycharmProjects\python\hackathon\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [79]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]
	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return np.array(bag)

In [80]:
#chat interface
def chat():
    
	print("Start Talking with the Bot (type quit to stop!)")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = np.argmax(results) # returns the index of the maximum element in a 1-d array
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in dataset["data"]:
				if tg['tag'] == tag:
					responses = tg['responses']
			print(random.choice(responses))
			print("\n")

		else:
			print("I didn't get that, please try again ! ")

In [81]:
#start chatting
chat()

Start Talking with the Bot (type quit to stop!)
You: glames all around me
Please consider the following measures- 
 1. Stay informed and stay updated about the flood warning and evacuation orders issued by the local authorities, and follow their instructions.
 2. If you are in a flood-prone area, be prepared to evacuate immediately.
 3. If you are at home, move to the highest point in your home.
 4. Turn off all electricity, gas, and water supplies if instructed to do so by the authorities.
 5. Do not try to move too much water by yourself. Wait for professional help. 



You: I AM DROWNING
I didn't get that, please try again ! 
You: I am choking
Please consider the following measures- 
 1. Stay calm and do not panic 
 2. Sound the fire alarm and shout to alert others in the house 
 3. Call the local fire department immediately by dialing 101 or the emergency number in your area. Provide them with your address and details of the fire. 
 4. Close all doors and windows to contain the fir